In [1]:
from multiprocessing import Pool
from joblib import Parallel, delayed
import multiprocessing
import pandas as pd
import re


data = pd.read_csv('Corea.csv', engine='python')

ADRframe=data[["#0Mid","14agree-disagree"]]

In [2]:
ADRframe.head(10)

,#0Mid,14agree-disagree
0,18006,NaN
1,18031,1.0
2,18086,1.0
3,18087,1.0
4,18106,1.0
5,18124,1.0
6,18145,NaN
7,18147,-1.0
8,18160,1.0
9,18179,-1.0


In [3]:
Lexframe = pd.read_csv('LexRateAWS.csv', engine='python')

In [4]:
Lexframe.head(10)

,cellaverage,mid
0,0.050000,18006
1,0.000000,18031
2,-0.089744,18086
3,0.142857,18087
4,0.208333,18106
5,0.361111,18124
6,0.000000,18145
7,-0.093750,18147
8,0.229167,18160
9,0.050000,18179


In [5]:
merged=pd.merge(Lexframe, ADRframe, left_on='mid', right_on='#0Mid')


In [6]:
merged.head(5)

,cellaverage,mid,#0Mid,14agree-disagree
0,0.050000,18006,18006,NaN
1,0.000000,18031,18031,1.0
2,-0.089744,18086,18086,1.0
3,0.142857,18087,18087,1.0
4,0.208333,18106,18106,1.0


In [7]:
merged=merged.drop('mid', axis=1)

In [8]:
merged.head(5)

,cellaverage,#0Mid,14agree-disagree
0,0.050000,18006,NaN
1,0.000000,18031,1.0
2,-0.089744,18086,1.0
3,0.142857,18087,1.0
4,0.208333,18106,1.0


In [9]:
mergedOrd=merged[["#0Mid","cellaverage","14agree-disagree"]]

In [10]:
mergedOrd.head()

,#0Mid,cellaverage,14agree-disagree
0,18006,0.050000,NaN
1,18031,0.000000,1.0
2,18086,-0.089744,1.0
3,18087,0.142857,1.0
4,18106,0.208333,1.0


In [11]:
mergedOrd.to_csv("AWSmerg.csv")

In [12]:
mergedOrd=mergedOrd.sort_values(by='#0Mid')
mergedOrd.head(10)

,#0Mid,cellaverage,14agree-disagree
0,18006,0.050000,NaN
1,18031,0.000000,1.0
2,18086,-0.089744,1.0
3,18087,0.142857,1.0
4,18106,0.208333,1.0
5,18124,0.361111,1.0
6,18145,0.000000,NaN
7,18147,-0.093750,-1.0
8,18160,0.229167,1.0
9,18179,0.050000,-1.0


In [61]:
alist=[]
for index, cell in mergedOrd.iterrows():
    if (cell["cellaverage"])<=(-0.1):
        alist.append(-1)
    if (cell["cellaverage"])>=(0.1):
        alist.append(1)
    else:
        alist.append(0)



In [62]:
row1=pd.DataFrame()
row1["ADRpred"]=alist
output=pd.concat([mergedOrd, row1],axis=1)
output.head(5)

,#0Mid,cellaverage,14agree-disagree,ADRpred
0,18006.0,0.050000,NaN,0
1,18031.0,0.000000,1.0,0
2,18086.0,-0.089744,1.0,0
3,18087.0,0.142857,1.0,1
4,18106.0,0.208333,1.0,1


In [63]:
output=output.dropna()
output.head(10)

,#0Mid,cellaverage,14agree-disagree,ADRpred
1,18031.0,0.000000,1.0,0
2,18086.0,-0.089744,1.0,0
3,18087.0,0.142857,1.0,1
4,18106.0,0.208333,1.0,1
5,18124.0,0.361111,1.0,1
7,18147.0,-0.093750,-1.0,0
8,18160.0,0.229167,1.0,1
9,18179.0,0.050000,-1.0,0
10,18189.0,0.266667,1.0,1
11,18191.0,0.055556,-1.0,0


In [64]:
from sklearn import metrics
metrics.mean_absolute_error(output["cellaverage"], output["14agree-disagree"])

0.90659711843018154

In [65]:
metrics.accuracy_score(output["14agree-disagree"],output["ADRpred"], )

0.27751834556370913

In [43]:
output.to_csv("preBinary.csv", index=False)

In [45]:
output2 = pd.read_csv('postBinary.csv', engine='python')


In [46]:
metrics.mean_absolute_error(output2["cellaverage"], output2["14agree-disagree"])

0.97585843873036082

In [47]:
metrics.accuracy_score(output2["14agree-disagree"],output2["ADRpred"], )

0.50743099787685775